## Imports

In [10]:
import cv2
import os
import re
import glob
import numpy as np
from PIL import Image
from zipfile import ZipFile
from matplotlib import pyplot as plt
from keras.preprocessing import image
from keras.models import model_from_json
from keras.models import Model, Sequential
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation

# Data preparation

In [6]:
def read_images(filename,preprocess = True):
    
    filename = filename
    
    imgs = []
    
    img_names = []
    
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    
    # read images after extracting zip file in RAM
    with ZipFile(filename) as archive:
        for count, entry in enumerate(archive.infolist()):
            with archive.open(entry) as file:
                if(file.name.endswith("jpg") or file.name.endswith("png")):
                    
                    # convert to grayscale
                    img = np.array(Image.open(file).convert('L'))

                    if(preprocess):
               
                        img = clahe.apply(img)

                    imgs.append(img)
                    
                    img_names.append(file.name)

    return imgs,img_names

In [7]:
template_imgs = []

template_img_names = []

test_imgs = []

test_img_names = []

bright_imgs = []

bright_img_names = []

contrast_imgs = []

contrast_img_names = []

rotation_imgs = []

rotation_img_names = []

for item in os.listdir():
    
    if(item.endswith("zip")):
        
        if(item == "BDD.zip"):
            
            tmp,tmp1 = read_images(item)

            template_imgs += tmp

            template_img_names += tmp1

        elif (item=="Test.zip") :

            tmp,tmp1 = read_images(item)

            test_imgs += tmp

            test_img_names += tmp1
            
        elif (item=="Contrast.zip") :

            tmp,tmp1 = read_images(item)

            contrast_imgs += tmp

            contrast_img_names += tmp1
            
        elif (item=="Rotation.zip") :

            tmp,tmp1 = read_images(item)

            rotation_imgs += tmp

            rotation_img_names += tmp1
            
        elif (item=="Brightness.zip"):

            tmp,tmp1 = read_images(item)

            bright_imgs += tmp

            bright_img_names += tmp1

contrast_imgs=np.array(contrast_imgs) 
bright_imgs=np.array(bright_imgs) 
rotation_imgs=np.array(rotation_imgs) 
test_imgs = np.array(test_imgs)
template_imgs = np.array(template_imgs)

In [8]:
test_imgs.shape,template_imgs.shape,contrast_imgs.shape,rotation_imgs.shape,bright_imgs.shape

((35, 250, 250),
 (70, 250, 250),
 (35, 250, 250),
 (35, 250, 250),
 (35, 250, 250))

# Evaluation

## Siamese network

In [9]:
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

Download the weights by executing this cell. It requires cURL installed, if you don't have it, download the file from this <a href="https://drive.google.com/file/d/1EjKFfz5sOUaZbpHgOo1FBLAUHSQMozp2/view?fbclid=IwAR3Z9jWs9CTAtvxXvweDH19W5XuEiIEb-tQRDHnZFATxAHvVaI-w0cFEc-s">link</a>

In [16]:
if('vgg_face_weights.h5' not in os.listdir()):
    !curl 'https://doc-0c-8s-docs.googleusercontent.com/docs/securesc/irhn5id42n45amehdukhdmveaa64qdca/pcjrmsabniqpn7f43f1hc29oic5su5o8/1546797600000/03047265511933935203/12338030662361016533/1EjKFfz5sOUaZbpHgOo1FBLAUHSQMozp2?e=download&nonce=3e2lits3ba8oo&user=12338030662361016533&hash=r0hqrcivs4pmu08iiludccr8qq8ubbv9' -H 'authority: doc-0c-8s-docs.googleusercontent.com' -H 'upgrade-insecure-requests: 1' -H 'user-agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36' -H 'accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8' -H 'x-client-data: CIa2yQEIprbJAQipncoBCKijygEIv6fKAQiQqMoBCOKoygEY+aXKAQ==' -H 'referer: https://drive.google.com/uc?id=1EjKFfz5sOUaZbpHgOo1FBLAUHSQMozp2&export=download' -H 'accept-encoding: gzip, deflate, br' -H 'accept-language: en-US,en;q=0.9' -H 'cookie: AUTH_7ncsoht60h5nj3290k7ksjvs3qm9agnm_nonce=3e2lits3ba8oo; NID=146=2m64YwK1FU_-wWYEr0N6qIT8KCPul94SVvGNE1YFUODGL-9ZZYgtf0PjHL_TrgeguaQjrWszdcM8aTlORHFViPx5R3pkEaM2uaQDIPyQkNY-ns5aQ0_77DSWxzvulfHWNxRIonT9b28IBORFdd1Cb8nPngy3F8nCQarRxGO2hNM' --compressed --output vgg_face_weights.h5

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  553M    0  553M    0     0  69.3M      0 --:--:--  0:00:07 --:--:-- 77.3M


In [17]:
model.load_weights('vgg_face_weights.h5')

In [18]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

In [19]:
def findCosineSimilarity(source_representation, test_representation):
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))

def findEuclideanDistance(source_representation, test_representation):
    euclidean_distance = source_representation - test_representation
    euclidean_distance = np.sum(np.multiply(euclidean_distance, euclidean_distance))
    euclidean_distance = np.sqrt(euclidean_distance)
    return euclidean_distance

In [20]:
vgg_face_descriptor = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)

In [21]:
epsilon = 0.25
epsilon1=50

def verifyFace(img1, img2):
    img1_representation = vgg_face_descriptor.predict(preprocess_image(img1))[0,:]
    img2_representation = vgg_face_descriptor.predict(preprocess_image(img2))[0,:]
    
    cosine_similarity = findCosineSimilarity(img1_representation, img2_representation)
    euclidean_distance = findEuclideanDistance(img1_representation, img2_representation)
    
    print("Cosine similarity: ",cosine_similarity)
    print("Euclidean distance: ",euclidean_distance)
    
    if(cosine_similarity < epsilon) and (euclidean_distance < epsilon1):
        print("verified... they are same person")
    else:
        print("unverified! they are not same person!")
    
    f = plt.figure()
    f.add_subplot(1,2, 1)
    plt.imshow(image.load_img(img1))
    plt.xticks([]); plt.yticks([])
    f.add_subplot(1,2, 2)
    plt.imshow(image.load_img(img2))
    plt.xticks([]); plt.yticks([])
    plt.show(block=True)
    print("-----------------------------------------")

## SIFT

## Minimum Pair Distance (MDP)

In [22]:
def MDP(img1, img2):

    sift = cv2.xfeatures2d.SIFT_create()
    
    kp1, des1 = sift.detectAndCompute(img1,None)

    kp2, des2 = sift.detectAndCompute(img2,None)

    # create BFMatcher object
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

    # Match descriptors.
    matches = bf.match(des1,des2)

    min_dist = np.min([matches[i].distance for i in range(len(matches))])
    
    return min_dist

## Regular Grid (RG)
_We will not use this method for comparison_

In [92]:
def RG(img1, img2):
    
    x,y = img1.shape
    
    x = np.int(x/4)
    
    y = np.int(y/2)
    
    i = 0
    
    min_dist = []
    
    while(x+i < img1.shape[0]):
        
        j = 0
        
        while(y+j < img1.shape[1]):
            
            min_dist.append(MDP(img1[i:x + i,j:y+j],img2[i:x + i,j:y+j]))
            
            
            j = np.int(j + 0.75*y)
            
        i = np.int(i +0.75*x)    
            
    return np.mean(min_dist)

## Results

__Siamese network__

In [23]:
threshold = 0.25
good_matches = 0
bad_matches = 0
for img_name in zip(test_img_names):
    img1 = vgg_face_descriptor.predict(preprocess_image(img_name[0]))[0,:]
    for name in zip(template_img_names):
        img2 = vgg_face_descriptor.predict(preprocess_image(name[0]))[0,:]
        if((findCosineSimilarity(img1,img2) <= threshold and img_name[0].split("/")[-1][:-9] == name[0].split("/")[-1][:-9]) or (findCosineSimilarity(img1,img2) > threshold and img_name[0].split("/")[-1][:-9] != name[0].split("/")[-1][:-9])):
            good_matches +=1
            
        else:
            bad_matches +=1

FileNotFoundError: [Errno 2] No such file or directory: 'Image2/Test/Abid_Hamid_Mahmud_Al-Tikriti_0002.jpg'

In [49]:
good_matches/(good_matches+bad_matches),bad_matches/(good_matches+bad_matches)

(0.9548611111111112, 0.04513888888888889)

__SIFT__

In [24]:
threshold = 30
good_matches = 0
bad_matches = 0
for img,img_name in zip(test_imgs,test_img_names):
    
    for i,name in zip(template_imgs,template_img_names):
        
        if((MDP(img,i) <= 30 and img_name.split("/")[-1][:-9] == name.split("/")[-1][:-9]) or (MDP(img,i) > 30 and img_name.split("/")[-1][:-9] != name.split("/")[-1][:-9])):
            good_matches +=1
            
        else:
#             print(img_name,name,MDP(img,i))
            bad_matches +=1

In [25]:
good_matches/(good_matches+bad_matches),bad_matches/(good_matches+bad_matches)

(0.9714285714285714, 0.02857142857142857)

<div class="alert alert-success">

<h2>  Resistance to the Change of Contrast</h2>
</div>

## Siamese network

In [53]:
from PIL import Image


def change_contrast(img, level):

    img = Image.open(img)
    img.load()

    factor = (259 * (level+255)) / (255 * (259-level))
    for x in range(img.size[0]):
        for y in range(img.size[1]):
            color = img.getpixel((x, y))
            new_color = tuple(int(factor * (c-128) + 128) for c in color)
            img.putpixel((x, y), new_color)

    return img

In [58]:
#for file in glob.glob("Image2/Test/*"):
 #   result=change_contrast(file, 100)
  #  result.save('Image2/Contrast/'+file[12:])

In [76]:
threshold = 0.25
good_matches = 0
bad_matches = 0
for img_name in zip(contrast_img_names):
    img1 = vgg_face_descriptor.predict(preprocess_image(img_name[0]))[0,:]
    for name in zip(template_img_names):
        img2 = vgg_face_descriptor.predict(preprocess_image(name[0]))[0,:]
        if((findCosineSimilarity(img1,img2) <= threshold and img_name[0].split("/")[-1][:-9] == name[0].split("/")[-1][:-9]) or (findCosineSimilarity(img1,img2) > threshold and img_name[0].split("/")[-1][:-9] != name[0].split("/")[-1][:-9])):
            good_matches +=1
            
        else:
#             print(img_name,name,MDP(img,i))
            bad_matches +=1

In [77]:
good_matches/(good_matches+bad_matches),bad_matches/(good_matches+bad_matches)

(0.9522569444444444, 0.04774305555555555)

__SIFT__

In [106]:
threshold = 30
good_matches = 0
bad_matches = 0
for img,img_name in zip(contrast_imgs,contrast_img_names):
    
    for i,name in zip(template_imgs,template_img_names):
        
        if((MDP(img,i) <= 30 and img_name.split("/")[-1][:-9] == name.split("/")[-1][:-9]) or (MDP(img,i) > 30 and img_name.split("/")[-1][:-9] != name.split("/")[-1][:-9])):
            good_matches +=1
            
        else:
            bad_matches +=1

In [107]:
good_matches/(good_matches+bad_matches),bad_matches/(good_matches+bad_matches)

(0.9583333333333334, 0.041666666666666664)

<div class="alert alert-success">

<h2>  Resistance to  Rotation</h2>
</div>

__Siamese network__

In [78]:
from PIL import Image
#for file in  glob.glob("Image2/Test/*"):
 #   im = Image.open(file)
  #  im1=im.rotate(30)
   # im1.save("Image2/Rotation/"+file[12:])

In [81]:
threshold = 0.25
good_matches = 0
bad_matches = 0
for img_name in zip(rotation_img_names):
    img1 = vgg_face_descriptor.predict(preprocess_image(img_name[0]))[0,:]
    for name in zip(template_img_names):
        img2 = vgg_face_descriptor.predict(preprocess_image(name[0]))[0,:]
        if((findCosineSimilarity(img1,img2) <= threshold and img_name[0].split("/")[-1][:-9] == name[0].split("/")[-1][:-9]) or (findCosineSimilarity(img1,img2) > threshold and img_name[0].split("/")[-1][:-9] != name[0].split("/")[-1][:-9])):
            good_matches +=1
            
        else:
#             print(img_name,name,MDP(img,i))
            bad_matches +=1

In [82]:
good_matches/(good_matches+bad_matches),bad_matches/(good_matches+bad_matches)

(0.9661458333333334, 0.033854166666666664)

__SIFT__

In [108]:
threshold = 30
good_matches = 0
bad_matches = 0
for img,img_name in zip(rotation_imgs,rotation_img_names):
    
    for i,name in zip(template_imgs,template_img_names):
        
        if((MDP(img,i) <= 30 and img_name.split("/")[-1][:-9] == name.split("/")[-1][:-9]) or (MDP(img,i) > 30 and img_name.split("/")[-1][:-9] != name.split("/")[-1][:-9])):
            good_matches +=1
            
        else:
            bad_matches +=1

In [109]:
good_matches/(good_matches+bad_matches),bad_matches/(good_matches+bad_matches)

(0.9583333333333334, 0.041666666666666664)

<div class="alert alert-success">

<h2>  Resistance to brightness</h2>
</div>

## Siamese network

In [85]:
from PIL import Image, ImageEnhance 
#for file in  glob.glob("Image2/Test/*"):
 #   enhancer = ImageEnhance.Brightness(im)
  #  enhanced_im = enhancer.enhance(1.8)
   # enhanced_im.save("Image2/Brightness/"+file[12:])

In [88]:
threshold = 0.25
good_matches = 0
bad_matches = 0
for img_name in zip(bright_img_names):
    img1 = vgg_face_descriptor.predict(preprocess_image(img_name[0]))[0,:]
    for name in zip(template_img_names):
        img2 = vgg_face_descriptor.predict(preprocess_image(name[0]))[0,:]
        if((findCosineSimilarity(img1,img2) <= threshold and img_name[0].split("/")[-1][:-9] == name[0].split("/")[-1][:-9]) or (findCosineSimilarity(img1,img2) > threshold and img_name[0].split("/")[-1][:-9] != name[0].split("/")[-1][:-9])):
            good_matches +=1
            
        else:
#             print(img_name,name,MDP(img,i))
            bad_matches +=1

In [89]:
good_matches/(good_matches+bad_matches),bad_matches/(good_matches+bad_matches)

(0.9392361111111112, 0.06076388888888889)

## SIFT

In [110]:
threshold = 30
good_matches = 0
bad_matches = 0
for img,img_name in zip(bright_imgs,bright_img_names):
    
    for i,name in zip(template_imgs,template_img_names):
        
        if((MDP(img,i) <= 30 and img_name.split("/")[-1][:-9] == name.split("/")[-1][:-9]) or (MDP(img,i) > 30 and img_name.split("/")[-1][:-9] != name.split("/")[-1][:-9])):
            good_matches +=1
            
        else:
#             print(img_name,name,MDP(img,i))
            bad_matches +=1

In [111]:
good_matches/(good_matches+bad_matches),bad_matches/(good_matches+bad_matches)

(0.9583333333333334, 0.041666666666666664)